In [ ]:
'''
purpose: 
    know the pros and cons of different classifier

acitons: 
    use different kinds of classifiers as the model
    classifiers includes perceptual, kNN, logistic regression, SVM, EM, baysian, HMM
                         adaboost, gbdt, xgboost, rf
                         deep learning models (DNN, CNN, RNN)
'''

In [1]:
# 以下代码的目的在于解决【中文字符】问题
import sys
stdout = sys.stdout
# 新版sys, 需要先reload(sys)再进行sys.setdefaultencoding操作
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
import os
# 设置工作目录
path = os.path.dirname(os.path.abspath('classifier_estimate.ipynb'))
os.chdir(path)

In [3]:
# 常用module
import pandas as pd 
import numpy as np
from imp import reload
from sklearn.externals import joblib

# machine learning
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
def my_custom_scorer(y_true, y_pred):
    return recall_score(y_true, y_pred, average='micro')

# local
import data_extract_X as dx
import data_extract_battle as db
import data_clean as dc

# encoding solver
import sys
stdout = sys.stdout
reload(sys) # 新版sys, 需要先reload(sys)再进行sys.setdefaultencoding操作
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

# basic 
import numpy as np
import time

# data analysis 
import pandas as pd 

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.utils import resample
from sklearn.preprocessing import add_dummy_feature
from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation
from collections import Counter

# local

In [4]:
### 第一步：数据的基本信息
reload(dc)
# And now to load scaler
X = pd.read_csv("output/extract_result.csv", index_col=0).drop(['actions'], axis=1).drop_duplicates('userid', keep='first')
y = pd.read_csv("output/new_3days_y_2018-03-24.csv", index_col=0).drop_duplicates('full_id', keep='first')
dropped_col = ['favorite', 'city_first', 'fromchannel_op_channel', 'fromchannel_iOS_plat', 'fromchannel_badchannel', 'completed_info', 'os_Android']

def preprocessing_df(data, y, dropped_col):
    # copy
    X = data.copy()
    
    # 导入本地数据清洗库
    import data_clean as dc
    dc_dc = dc.data_clean()
    
    # 处理X 中的 数字变量
    columns_filterd_one = ['SignIn', 'unsub', 'sub', 'purchase', 'share', 'favorite']
    columns_filterd_second = ['read']
    X = dc_dc.X_numberic_outlier(X, columns_filterd_one, .999, columns_filtered_second = columns_filterd_second, \
                       critical_percent_second = .99, case_one=True)
    
    # 处理X中的分类变量
    X_train, X_test, y_train, y_test = dc_dc.X_object_preprocessing(X,y)
    X = X_train.append(X_test, ignore_index = True)
    y = y_train.append(y_test, ignore_index = True)
    
    # get dummies 
    X_userid = X.loc[:, 'userid']
    X_dummies = pd.get_dummies(X.drop(['userid', 'date'], axis=1))
    
    # 特征工程部分
    X_dummies = X_dummies.drop(dropped_col, axis=1)
    
    # prepare for modelling 
    # 归一化   
    #model_columns = np.loadtxt('output/model_columns', dtype='S')
    #formal_df = pd.DataFrame([], columns=model_columns)
    #formal_df = formal_df.append(X_dummies, ignore_index=True).fillna(0.0)
    #scaler = joblib.load('output/scaler.save') 
    #X_scaler = scaler.transform(formal_df[model_columns])  
    
    # output 
    return X_userid, X_dummies, y
    #return formal_df, scaler
X_userid, X_dummies, y = preprocessing_df(X, y, dropped_col)

In [23]:
print "X shape is {}, y shape is {}".format(X_dummies.shape, y.shape)

X shape is (66858, 21), y shape is (66858,)


In [25]:
X_dummies.describe()

,SignIn,weekend,coin,lucky_money,unsub,sub,default_buyer,today_read,purchase,share,...,path1,path2,os_iOS,city_others,city_second,gender_female,gender_male,status_student,status_worker,fromchannel_goodchannel
count,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,...,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000,66858.000000
mean,0.295327,0.329609,0.064300,0.066858,0.258398,0.556059,12.742364,5.371952,0.162778,0.160759,...,3.083101,0.039756,0.508466,0.329983,0.216249,0.647028,0.352972,0.491280,0.508720,0.394179
std,0.456193,0.470075,0.245289,0.249778,1.467620,2.403917,1.455490,5.310532,0.479027,0.740255,...,7.963045,0.229205,0.499932,0.470210,0.411689,0.477898,0.477898,0.499928,0.499928,0.488677
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,3.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,14.000000,11.000000,0.000000,0.000000,...,2.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,15.000000,55.000000,16.000000,15.000000,6.000000,13.000000,...,143.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [2]:
### 简单模型
1. kNN
2. perceptual 
3. logistic regression 
4. EM
5. SVM

In [5]:
# split trian & test dataset 
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.3, random_state=2)

# scale 
# load saved scaler
model_columns = np.loadtxt('output/model_columns', dtype='S')
scaler = joblib.load('output/scaler.save')
# transform
X_scaler = scaler.transform(X_train[model_columns]) 
X_test_scaler = scaler.transform(X_test[model_columns]) 

In [ ]:
from sklearn.metrics import f1_score
X_train_scaler = X_scaler.copy()
y_train = y_train.copy()
classifier = LogisticRegression(penalty='l2', dual=False, tol=0.1, C=8.0, fit_intercept=True, intercept_scaling=1, random_state=3, solver='sag', verbose=0, warm_start=False, n_jobs=4)
def my_custom_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')
scoring = make_scorer(my_custom_scorer)
def estimator_train(classifier, X_train_scaler, y_train):
    para_grid = dict(
        penalty = ['l1'], 
        ##tol = [0.1, 0.05, 0.001, 0.0005, 0.00001], 
        tol = [0.0005], 
        ##C = [20, 10, 2, 1], 
        C = [2], 
        solver = ['liblinear'], 
        max_iter = [1000000])
    clf = GridSearchCV(estimator=classifier, param_grid=para_grid, scoring=scoring, fit_params=None, n_jobs=4, iid=True, refit=True, cv=10, verbose=0, pre_dispatch='1',  error_score='raise', return_train_score='warn')
    clf.fit(X_train_scaler, y_train)
    print classification_report(y_train, clf.predict(X_train_scaler))
    return clf
final_clf = estimator_train(classifier, X_train_scaler, y_train)

In [7]:
pred_labels = final_clf.predict(X_test_scaler)
print classification_report(y_test, pred_labels)

             precision    recall  f1-score   support

          0       0.78      0.96      0.86     14970
          1       0.63      0.20      0.30      5088

avg / total       0.74      0.77      0.72     20058



In [8]:
print final_clf.best_estimator_

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=4,
          penalty='l1', random_state=3, solver='liblinear', tol=0.0005,
          verbose=0, warm_start=False)


In [4]:
#import xgboost

SyntaxError: invalid syntax (<ipython-input-4-492d8868e019>, line 2)